In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium

# Overview

(1). Get Health region shapefiles and merge data using health region unique id (HR_UID)

(2). Get GMM labels and merge these datasets to the dataset curated from (1) for later visualization

(3). Creating maps of Canadian Health region with different colors based on the GMM labels and information of healthcare facilities included. The maps were made interactive with Folium library.

# Getting Health Region Shapefiles

## Getting Shapefiles 1

Due to inconsistencies between the health regions (HR) in our hospital bed availability and the HR in the shape files, we will extract the polygons of AB, BC, NW, YT, NU, PE, and NS from the Health_Region_Summaries.zip

Data source: https://resources-covid19canada.hub.arcgis.com/datasets/covid19canada::health-region-summaries/about

In [2]:
path_boundaries = "C:/Users/kimng/Desktop/AI4Good Project/Unzip files/NewHybridRegionalHeathBoundaries.shp"

map_HR = gpd.read_file(path_boundaries, encoding="utf-8")
#map_HR.head()

In [3]:
#Filter the dataset to get health region boundaries of interests
map_filtered = map_HR.query("Province=='AB' | Province=='BC'| Province=='PE'| Province=='YT' | Province=='NT'| Province=='NU'| Province=='MB' ")

#reset the coordinate system
map_filtered= map_filtered.to_crs('ESRI:102009') 

#Change health region unique ID (HR_UID) to numeric for later merging with health access data sets
map_filtered["HR_UID"] = pd.to_numeric(map_filtered["HR_UID"])

#Get the shape data data and exclude irrelevant columns from the orginal data set.
map_filtered = map_filtered[['HR_UID', 'geometry', 'Province']]

#Check the data again to ensure correct format
#map_filtered.head()

In [4]:
#Import number of hospital beds staffed and in operation (CIHI) data. This dataset includes HR_UID
# More information about this dataset can be found at 
# https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fwww.cihi.ca%2Fsites%2Fdefault%2Ffiles%2Fdocument%2Fbeds-staffed-and-in-operation-2020-2021-en.xlsx&wdOrigin=BROWSELINK

beds = pd.read_csv("C:/Users/kimng/Desktop/AI4Good Project/Cleaned data/beds_mapid.csv" )
#beds.head()

In [5]:
#display(beds.loc[beds.HR_UID.isna()])

beds = beds.dropna(subset=['HR_UID'])

#display(beds.loc[beds.HR_UID.isna()])

In [6]:
# Change HR_UID to integer to merge with the shape file (map_filtered) dataset
beds['HR_UID'] = beds['HR_UID'].apply(np.int64)

In [7]:
# Merge beds and map_filtered data sets based on HR_UID 
# and set healthy region shape as geometry variable from map_filtered dataset

beds_map1 = gpd.GeoDataFrame(map_filtered.merge(beds,how='left',on=['HR_UID']),geometry='geometry')
#beds_map1.head()

In [8]:
beds_map1 = beds_map1.to_crs('ESRI:102009') #set coordinate sys

## Getting shapefiles 2

We will extract the polygons of NS, NB, SK, NL, QC, ON, MB from the canada-eng.zip

Data source: https://www150.statcan.gc.ca/n1/pub/82-402-x/2009001/rg-eng.htm#arcinfo

More info about health regions: https://www150.statcan.gc.ca/n1/pub/82-402-x/2018001/hrpg-rsgh-eng.htm

In [9]:
filepath_NS = "C:/Users/kimng/Desktop/AI4Good Project/Unzip files/HR012b08z/HR012b08z.shp"
filepath_NB = "C:/Users/kimng/Desktop/AI4Good Project/Unzip files/HR013b08/HR013b08.shp" 
filepath_SK = "C:/Users/kimng/Desktop/AI4Good Project/Unzip files/HR047b08/HR047b08.shp" 
filepath_NL = "C:/Users/kimng/Desktop/AI4Good Project/Unzip files/HR010b08/HR010b08.shp" 
filepath_QC = "C:/Users/kimng/Desktop/AI4Good Project/Unzip files/HR024b08/HR024b08.shp" 
filepath_ON = "C:/Users/kimng/Desktop/AI4Good Project/Unzip files/HR035b08/HR035b08.shp" 

map_SK= gpd.read_file(filepath_SK, encoding="utf-8")
map_SK.rename(index=str, columns ={'HRUID2007':'HR_UID'}, inplace=True)
map_NB= gpd.read_file(filepath_NB, encoding="utf-8")
map_NB.rename(index=str, columns ={'HRUID2007':'HR_UID'}, inplace=True)
map_NS = gpd.read_file(filepath_NS, encoding="utf-8")
map_NS.rename(index=str, columns ={'HRUID2007':'HR_UID'}, inplace=True)
map_NL= gpd.read_file(filepath_NL, encoding="utf-8")
map_NL.rename(index=str, columns ={'HRUID2007':'HR_UID'}, inplace=True)
map_QC= gpd.read_file(filepath_QC, encoding="utf-8")
map_QC.rename(index=str, columns ={'HRUID2007':'HR_UID'}, inplace=True)
map_ON= gpd.read_file(filepath_ON, encoding="utf-8")
map_ON.rename(index=str, columns ={'HRUID2007':'HR_UID'}, inplace=True)

map_SK["HR_UID"] = pd.to_numeric(map_SK["HR_UID"])
map_SK = map_SK[['HR_UID', 'geometry']]
map_NB["HR_UID"] = pd.to_numeric(map_NB["HR_UID"])
map_NB = map_NB[['HR_UID', 'geometry']]
map_NS["HR_UID"] = pd.to_numeric(map_NS["HR_UID"])
map_NS = map_NS[['HR_UID', 'geometry']]
map_NL["HR_UID"] = pd.to_numeric(map_NL["HR_UID"])
map_NL = map_NL[['HR_UID', 'geometry']]
map_QC["HR_UID"] = pd.to_numeric(map_QC["HR_UID"])
map_QC = map_QC[['HR_UID', 'geometry']]
map_ON["HR_UID"] = pd.to_numeric(map_ON["HR_UID"])
map_ON = map_ON[['HR_UID', 'geometry']]


map2 = gpd.GeoDataFrame(pd.concat([map_NB, map_SK, map_NL, map_QC, map_ON, map_NS],ignore_index=True),  
                        geometry='geometry')
#map2.head()

In [10]:
# Merge beds and map2 datasets based on HR_UID 
# and set healthy region shape as geometry variable from map_filtered dataset

beds_map2 = gpd.GeoDataFrame(map2.merge(beds,how='left',on=['HR_UID']),geometry='geometry')
#beds_map2.head()

In [11]:
#reset coordinate system to make sure datasets containing geometry shapes are using the same crs.
beds_map2 = beds_map2.to_crs('ESRI:102009')

In [12]:

beds_map1= beds_map1.reset_index()
beds_map2= beds_map2.reset_index()

#merge beds_map1 and beds_map2 to get a dataset with all health region shape information and associating no. of hospital beds info
beds_map_all = gpd.GeoDataFrame(pd.concat([beds_map1, beds_map2],ignore_index=True),  geometry='geometry')


In [13]:
beds_map_all['HR_UID'] = beds_map_all['HR_UID'].astype(str)
#beds_map_all = beds_map_all.set_index('HR_UID')
#beds_map_all.shape

# Getting GMM labels and merging with map shapefiles

**Prepare Mental Health GMM dataset for interactive map**

In [42]:
#Get Mental Health clusters 
path_labels = "C:/Users/kimng/Desktop/AI4Good Project/GMM_MentalHealthcare_labels.csv"
MH_labels = pd.read_csv(path_labels, encoding="utf-8")
#MH_labels.info()

In [43]:
MH_labels['HR_UID'] = MH_labels['HR_UID'].apply(np.int64)
beds_map_all['HR_UID'] = beds_map_all['HR_UID'].apply(np.int64)
MH_labels['labels'] = MH_labels['labels'].astype(str)

In [54]:
#Merge dataset containing Mental Health GMM labels and dataset containing health region geospatial shapes curated earlier
MH_label_map = gpd.GeoDataFrame(beds_map_all.merge(MH_labels,how='left',on=['HR_UID']), geometry = 'geometry')

#Get relevant data only

cols = ['geometry', 'Province/territory','Health region' , 'MHBed-to-100,000_LowPerceivedMH', 
        'Psychiatrist-to-100,000_LowPerceivedMH', 'labels',
       'prob_groupA', 'prob_groupB', 'prob_groupC', 'prob_groupD', 'prob_groupE']
#MH_label_map = MH_label_map.reindex(cols)

MH_label_map = MH_label_map[cols]

#Rename columns
MH_label_map.rename(columns = {
                               'regular_provider_p':'Population that has a regular healthcare provider (%)',
                               'MHBed-to-100,000_LowPerceivedMH': 'No. of mental health beds per 100,000 low perceived mental health population',
                               'Psychiatrist-to-100,000_LowPerceivedMH': 'No. of psychiatrists per 100,000 low perceived mental health population',
                               'labels': 'Mental Healthcare Access Needs',
                              'prob_groupA': 'Probability of region belonging to Group A',
                              'prob_groupB': 'Probability of region belonging to Group B',
                              'prob_groupC': 'Probability of region belonging to Group C',
                              'prob_groupD': 'Probability of region belonging to Group D',
                              'prob_groupE': 'Probability of region belonging to Group E',
                              },
                    inplace = True)
#MH_label_map.info()

In [55]:
MH_label_map.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 81 entries, 0 to 80
Data columns (total 11 columns):
 #   Column                                                                        Non-Null Count  Dtype   
---  ------                                                                        --------------  -----   
 0   geometry                                                                      81 non-null     geometry
 1   Province/territory                                                            77 non-null     object  
 2   Health region                                                                 77 non-null     object  
 3   No. of mental health beds per 100,000 low perceived mental health population  72 non-null     float64 
 4   No. of psychiatrists per 100,000 low perceived mental health population       72 non-null     float64 
 5   Mental Healthcare Access Needs                                                72 non-null     object  
 6   Probability of regio

In [56]:
MH_label_map.loc[MH_label_map.isnull().any(axis=1)]

,geometry,Province/territory,Health region,"No. of mental health beds per 100,000 low perceived mental health population","No. of psychiatrists per 100,000 low perceived mental health population",Mental Healthcare Access Needs,Probability of region belonging to Group A,Probability of region belonging to Group B,Probability of region belonging to Group C,Probability of region belonging to Group D,Probability of region belonging to Group E
2,"MULTIPOLYGON (((469729.469 5209339.916, 468778...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,"POLYGON ((-343567.818 2010269.533, -344067.304...",SK,Mamawetan Churchill River Health Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,"POLYGON ((-571701.613 2032207.531, -572641.238...",SK,Keewatin Yatthé Health Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,"POLYGON ((-334535.915 2141332.098, -334536.634...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,"MULTIPOLYGON (((1132920.661 1301900.754, 11329...",QC,Nord-du-Québec Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,"MULTIPOLYGON (((1092195.603 2023189.937, 10923...",QC,Nunavik Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,"MULTIPOLYGON (((1327651.254 1161577.929, 13277...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,"MULTIPOLYGON (((2449488.676 1113654.954, 24495...",NS,IWK Health Centre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,"MULTIPOLYGON (((2351788.235 906245.725, 235180...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
#Rename the GMM labels

MH_label_map.loc[MH_label_map['Mental Healthcare Access Needs'] == "0", 'Mental Healthcare Access Needs'] = 'Group A: Average No. psychiatrists, Average No. available beds'

MH_label_map.loc[MH_label_map['Mental Healthcare Access Needs'] == "1", 'Mental Healthcare Access Needs']= 'Group B: Below-average No. psychiatrists, Below-average No. available beds'

MH_label_map.loc[MH_label_map['Mental Healthcare Access Needs'] == "2", 'Mental Healthcare Access Needs']= 'Group C: Below-average No. psychiatrists, Highest No. available beds'

MH_label_map.loc[MH_label_map['Mental Healthcare Access Needs'] == "3", 'Mental Healthcare Access Needs']= 'Group D: Above-average No. psychiatrists, Above-average No. available beds'

MH_label_map.loc[MH_label_map['Mental Healthcare Access Needs'] == "4", 'Mental Healthcare Access Needs']= 'Group E: Critically below-average No. psychiatrists, below-average No. available beds'

In [59]:
#Check new labels and make sure there is no NA

MH_label_map['Mental Healthcare Access Needs'].unique()


array(['Group E: Critically below-average No. psychiatrists, below-average No. available beds',
       nan,
       'Group B: Below-average No. psychiatrists, Below-average No. available beds',
       'Group A: Average No. psychiatrists, Average No. available beds',
       'Group C: Below-average No. psychiatrists, Highest No. available beds',
       'Group D: Above-average No. psychiatrists, Above-average No. available beds'],
      dtype=object)

In [24]:
##Change NaN labels to a string value so that the area will be mapped despite missingness
#MH_label_map['labels'].fillna('Missing Data', inplace = True)
##Note this is not needed if geopandas is used for chorophleth

**Prepare Overall healthcare GMM labels dataset for interactive map**

In [25]:
#overall healthcare labels
HC_labels = pd.read_csv("C:/Users/kimng/Desktop/AI4Good Project/GMM_Healthcare_labels.csv", encoding="utf-8")

HC_labels['HR_UID'] = HC_labels['HR_UID'].apply(np.int64)
beds_map_all['HR_UID'] = beds_map_all['HR_UID'].apply(np.int64)
HC_labels['labels'] = HC_labels['labels'].astype(str)

HC_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 26 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Unnamed: 0                              75 non-null     int64  
 1   index                                   75 non-null     int64  
 2   Health region_x                         75 non-null     object 
 3   Province/territory_x                    75 non-null     object 
 4   HR_UID                                  75 non-null     int64  
 5   regular_provider_p                      75 non-null     float64
 6   TotalBed-to-100,000_LowPerceivedHealth  75 non-null     float64
 7   Physician-to–100,000 population ratio   75 non-null     float64
 8   labels                                  75 non-null     object 
 9   index.1                                 75 non-null     int64  
 10  prob_groupA                             75 non-null     float64


In [26]:
#HC_labels[[v, 'prob_groupF', 'prob_groupG']] = HC_labels[['prob_groupA', 'prob_groupB', 'prob_groupC', 'prob_groupD', 'prob_groupE', 'prob_groupF', 'prob_groupG']]*100

In [27]:
#HC_labels.head()

In [28]:

HC_label_map = gpd.GeoDataFrame(beds_map_all.merge(HC_labels,how='left',on=['HR_UID']), geometry = 'geometry')

cols = ['geometry', 'Province/territory', 'Health region' , 
        'regular_provider_p', 'TotalBed-to-100,000_LowPerceivedHealth', 
        'Physician-to–100,000 population ratio', 'labels', 
        'prob_groupA', 'prob_groupB', 'prob_groupC', 'prob_groupD', 'prob_groupE',
        'prob_groupF', 'prob_groupG',
       'prob_max']

HC_label_map = HC_label_map[cols]

HC_label_map.rename(columns = {'regular_provider_p': 'Population that has a regular healthcare provider (%)',
                            'TotalBed-to-100,000_LowPerceivedHealth': 'Total no. of available beds per 100,000 low perceived health population',
                            'Physician-to–100,000 population ratio': 'Number of physicians per 100,000 population',
                             'labels': 'Overall Healthcare Access Needs',
                              'prob_groupA': 'Probability of region belonging to Group A',
                              'prob_groupB': 'Probability of region belonging to Group B',
                              'prob_groupC': 'Probability of region belonging to Group C',
                              'prob_groupD': 'Probability of region belonging to Group D',
                              'prob_groupE': 'Probability of region belonging to Group E',
                              'prob_groupF': 'Probability of region belonging to Group F',
                              'prob_groupG': 'Probability of region belonging to Group G'},
                    inplace = True)
HC_label_map.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 81 entries, 0 to 80
Data columns (total 15 columns):
 #   Column                                                                   Non-Null Count  Dtype   
---  ------                                                                   --------------  -----   
 0   geometry                                                                 81 non-null     geometry
 1   Province/territory                                                       77 non-null     object  
 2   Health region                                                            77 non-null     object  
 3   Population that has a regular healthcare provider (%)                    75 non-null     float64 
 4   Total no. of available beds per 100,000 low perceived health population  75 non-null     float64 
 5   Number of physicians per 100,000 population                              75 non-null     float64 
 6   Overall Healthcare Access Needs                             

In [29]:
HC_label_map.loc[HC_label_map['Overall Healthcare Access Needs'] == "0", 'Overall Healthcare Access Needs'] = 'Group A: Need for more hospital beds'

HC_label_map.loc[HC_label_map['Overall Healthcare Access Needs'] == "1", 'Overall Healthcare Access Needs'] = 'Group B: Urgent need for more physicians'

HC_label_map.loc[HC_label_map['Overall Healthcare Access Needs'] == "2", 'Overall Healthcare Access Needs'] = 'Group C: Need for more physicians and higher percentage of population with a regular healthcare provider'

HC_label_map.loc[HC_label_map['Overall Healthcare Access Needs'] == "3", 'Overall Healthcare Access Needs'] = 'Group D: Need for more physicians'

HC_label_map.loc[HC_label_map['Overall Healthcare Access Needs'] == "4", 'Overall Healthcare Access Needs'] = 'Group E: Need for more physicians, hospital beds, and percentage of population with a regular healthcare'

HC_label_map.loc[HC_label_map['Overall Healthcare Access Needs'] == "5", 'Overall Healthcare Access Needs'] = 'Group F: Need for more physicians and hospital beds'

HC_label_map.loc[HC_label_map['Overall Healthcare Access Needs'] == "6", 'Overall Healthcare Access Needs'] = 'Group G: Sufficient access'


In [30]:
#HC_label_map.rename(columns = {'Overall Healthcare Access Group' :'Overall Healthcare Access Needs'}, inplace = True)

In [31]:
HC_label_map['Overall Healthcare Access Needs'].unique()

array(['Group F: Need for more physicians and hospital beds',
       'Group C: Need for more physicians and higher percentage of population with a regular healthcare provider',
       nan, 'Group A: Need for more hospital beds',
       'Group G: Sufficient access',
       'Group B: Urgent need for more physicians',
       'Group E: Need for more physicians, hospital beds, and percentage of population with a regular healthcare',
       'Group D: Need for more physicians'], dtype=object)

In [32]:
HC_label_map.loc[HC_label_map.isnull().any(axis=1)]

,geometry,Province/territory,Health region,Population that has a regular healthcare provider (%),"Total no. of available beds per 100,000 low perceived health population","Number of physicians per 100,000 population",Overall Healthcare Access Needs,Probability of region belonging to Group A,Probability of region belonging to Group B,Probability of region belonging to Group C,Probability of region belonging to Group D,Probability of region belonging to Group E,Probability of region belonging to Group F,Probability of region belonging to Group G,prob_max
2,"MULTIPOLYGON (((469729.469 5209339.916, 468778...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,"POLYGON ((-334535.915 2141332.098, -334536.634...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,"MULTIPOLYGON (((1092195.603 2023189.937, 10923...",QC,Nunavik Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,"MULTIPOLYGON (((1327651.254 1161577.929, 13277...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,"MULTIPOLYGON (((2449488.676 1113654.954, 24495...",NS,IWK Health Centre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,"MULTIPOLYGON (((2351788.235 906245.725, 235180...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Interactive Map with hospital info

In [33]:
# path to shapefile of hospital locations
filepath = "C:/Users/kimng/Desktop/AI4Good Project/Unzip files/Open_Database_of_Healthcare_Facilities.shp"

# Read file using gpd.read_file()
map1 = gpd.read_file(filepath, encoding="utf-8")

map1.loc[map1['odhf_facil'] == "nursing and residential care facilities", 'odhf_facil'] = 'Nursing and residential care facilities' # correcting the category
#For simplicity, the rows with missing odhf_facil will be excluded from the map

#map1 = map1.dropna(subset=['odhf_facil'])
map1 = map1.to_crs(MH_label_map.crs)

In [34]:
#recategorize unclassified rows
map1.loc[[2072,2490,2490], "odhf_facil"] = 'Hospitals'

map1.loc[map1.odhf_facil.isna(), 'odhf_facil'] = 'Others'

map1.rename(columns = {'source_fac' :'Facility Information'}, inplace = True)

In [35]:
#canada = folium.Map(location = [61.066922, -107.9917071], 
#                  tiles="cartodbpositron", zoom_start = 4)

**Create interactive map for overall healthcare access**

In [62]:
import folium

# Create a Map instance

base = folium.Map(location = [61.066922, -107.9917071],
                    #location = [lat, long], # entercanada lat and lon
                    tiles="cartodbpositron",
                    zoom_start = 4, control_scale=True)

### create choropleth plot

HC_label_map.explore(m=base, #add layer to base map
                     column='Overall Healthcare Access Needs', # make choropleth based on "labels" column
                     tooltip="Health region", # show "Health region" value in tooltip (on hover)
                     popup=True, # show all values in popup (on click)
#                     fill_opacity = 'prob_max',
                     cmap= ['#85e1bb', 'orange', 'green', 'red',
                           'purple', 'pink', 'yellow'] , # use matplotlib colormap
                     style_kwds=dict(color="grey"), # use grey outline
                     missing_kwds={
                        "color": "grey",
                        "label": "Incomplete or unavailable data",
                    },
                     name = 'Overall Healthcare Access Needs')

# MH_label_map.explore(m=base, #add layer to base map
#                      column='Mental Healthcare Access Peer Group', # make choropleth based on "labels" column
#                      tooltip="Health region", # show "Health region" value in tooltip (on hover)
#                      popup=True, # show all values in popup (on click)
#                      cmap='Set1', # use "Set1" matplotlib colormap
#                      style_kwds=dict(color="grey"), # use grey outline
#                      missing_kwds={
#                         "color": "black",
#                         "edgecolor": "red",
#                         "hatch": "///",
#                         "label": "Missing values",
#                     },
#                      name = 'Mental Healthcare Access Groups')


### Create marker and clusters for each hospital type

from folium import plugins

map1 = map1.dropna(subset=['latitude'])
map1 = map1.dropna(subset=['longitude'])

Ambulatory1 = map1[map1.odhf_facil=='Ambulatory health care services']
Ambulatory = Ambulatory1[['latitude', 'longitude']]

hospital1 = map1[map1.odhf_facil=='Hospitals']
hospital = hospital1[['latitude', 'longitude']]

residential1 = map1[map1.odhf_facil=='Nursing and residential care facilities']
residential = residential1[['latitude', 'longitude']]

others1 = map1[map1.odhf_facil=='Others']
others = others1[['latitude', 'longitude']]

# name1 = Ambulatory1['Name']
# source_fac1 = Ambulatory1['source_fac']
# popups_list1 = list(zip(name1, source_fac1))
# popups1 = ["Name:{}<br>Facility:{}".format(Name, source_fac) for (Name, source_fac) in popups_list1]
plugins.MarkerCluster(Ambulatory,
                      control=True,
                      name = 'Ambulatory health care services',
#                     icon=folium.Icon(icon_color='red')
                      show = False, #disable the showing of the marker when map first loads
                     ).add_to(base)

# name2 = hospital1['Name']
# source_fac2 = hospital1['source_fac']
# popups_list2 = list(zip(name2, source_fac2))
# popups2 = ["Name:{}<br>Facility:{}".format(Name, source_fac) for (Name, source_fac) in popups_list2]
plugins.MarkerCluster(hospital,
                      control=True,
                      name = 'Hospitals',
#                     icon=folium.Icon(icon_color='blue')
                      show = False, #disable the showing of the marker when map first loads
                     ).add_to(base)

# name3 = residential1['Name']
# source_fac3 = residential1['source_fac']
# popups_list3 = list(zip(name3, source_fac3))
# popups3 = ["Name:{}<br>Facility:{}".format(Name, source_fac) for (Name, source_fac) in popups_list3]
plugins.MarkerCluster(residential,
                      control=True,
                      tooltip="Name",
                      name = 'Nursing and residential care facilities',
#                     icon=folium.Icon(icon_color='green')
                      show = False, #disable the showing of the marker when map first loads
                     ).add_to(base)

# name4 = others1['Name']
# source_fac4 = others1['source_fac']
# popups_list4 = list(zip(name4, source_fac4))
# popups4 = ["Name:{}<br>Facility:{}".format(Name, source_fac) for (Name, source_fac) in popups_list4]
plugins.MarkerCluster(others,
                      control=True,
                      name = 'Other facility',
                      tooltip="Name",
#                     icon=folium.Icon(icon_color='orange')
                      show = False, #disable the showing of the marker when map first loads
                     ).add_to(base)

## create marker for location of each facility type
map1[map1['odhf_facil'] == 'Hospitals'].explore(m = base, #pass the base map object
             marker_kwds=dict(radius=5, fill=True), 
             legend = True,
             color='#4654A3', popup = 'Facility Information',
             show = False, #disable the showing of the marker when map first loads
             tooltip="Name", name = 'Hospital Locations')


map1[map1['odhf_facil'] == 'Nursing and residential care facilities'].explore(m = base,
             marker_kwds=dict(radius= 5, fill=True),
             color='#3DD8F9', popup = 'Facility Information',
             show = False, #disable the showing of the marker when map first loads
             tooltip="Name", name = 'Nursing and residential care facility Locations')


map1[map1['odhf_facil'] == 'Ambulatory health care services'].explore(m = base,
             marker_kwds=dict(radius=5, fill=True),
             color='#FEB7C7',popup = 'Facility Information',
             show = False, #disable the showing of the marker when map first loads
             tooltip="Name", name = 'Ambulatory health care service locations')


map1[map1['odhf_facil'] == 'Others'].explore(m = base,
             marker_kwds=dict(radius=5, fill=True),
             color='red',popup = 'Facility Information',
             show = False, #disable the showing of the marker when map first loads
             tooltip="Name", name = 'Other facility location')


folium.TileLayer('openstreetmap', control=True).add_to(base)  # use folium to add alternative tiles
 
folium.LayerControl(collapsed = False).add_to(base)  # use folium to add layer control


## Saving the map as html

outfp = "OverallHealthcareAccessMap.html"
base.save(outfp)

**Create interactive map for mental/psychiatric healthcare access**

In [61]:
import folium

# Create a Map instance

base = folium.Map(location = [61.066922, -107.9917071],
                    #location = [lat, long], # entercanada lat and lon
                    tiles="cartodbpositron", zoom_start = 4, control_scale=True)

### create choropleth plot

# df_final.explore(m=base, #add layer to base map
#                      column='Overall Healthcare Access Peer Group', # make choropleth based on "labels" column
#                      tooltip="Health region", # show "Health region" value in tooltip (on hover)
#                      popup=True, # show all values in popup (on click)
#                      cmap='Set1', # use "Set1" matplotlib colormap
#                      style_kwds=dict(color="grey"), # use grey outline
#                      missing_kwds={
#                         "color": "black",
#                         "edgecolor": "red",
#                         "hatch": "///",
#                         "label": "Missing values",
#                     },
#                      name = 'Overall Healthcare Access Groups')

MH_label_map.explore(m=base, #add layer to base map
                     column='Mental Healthcare Access Needs', # make choropleth based on "labels" column
                     tooltip="Health region", # show "Health region" value in tooltip (on hover)
                     popup=True, # show all values in popup (on click)
                     cmap=['#85e1bb', 'orange', 'green', 'red',
                           'purple'],
                     style_kwds=dict(color="grey"), # use grey outline
                     missing_kwds={
                        "color": "gray",
                        "label": "Incomplete or unavailable data", #option for missing values
                    },
                     name = 'Mental Healthcare Access Needs')


### Create marker and clusters for each hospital type

from folium import plugins

map1 = map1.dropna(subset=['latitude'])
map1 = map1.dropna(subset=['longitude'])

Ambulatory1 = map1[map1.odhf_facil=='Ambulatory health care services']
Ambulatory = Ambulatory1[['latitude', 'longitude']]

hospital1 = map1[map1.odhf_facil=='Hospitals']
hospital = hospital1[['latitude', 'longitude']]

residential1 = map1[map1.odhf_facil=='Nursing and residential care facilities']
residential = residential1[['latitude', 'longitude']]

others1 = map1[map1.odhf_facil=='Others']
others = others1[['latitude', 'longitude']]

plugins.MarkerCluster(Ambulatory,
                      control=True,
                      name = 'Ambulatory health care services',
                      show = False, #disable the showing of the marker when map first loads
#                     icon=folium.Icon(icon_color='red')
                     ).add_to(base)


plugins.MarkerCluster(hospital,
                      control=True,
                      name = 'Hospitals',
                      show = False, #disable the showing of the marker when map first loads
#                     icon=folium.Icon(icon_color='blue')
                     ).add_to(base)

plugins.MarkerCluster(residential,
                      control=True,
                      tooltip="Name",
                      name = 'Nursing and residential care facilities',
                      show = False, #disable the showing of the marker when map first loads
#                     icon=folium.Icon(icon_color='green')
                     ).add_to(base)


plugins.MarkerCluster(others,
                      control=True,
                      name = 'Other facility',
                      tooltip="Name",
                      show = False, #disable the showing of the marker when map first loads
#                      icon=folium.Icon(icon_color='orange')
                     ).add_to(base)

## create marker for location of each facility type

# set up pop up infos

# name1 = Ambulatory1['Name']
# source_fac1 = Ambulatory1['source_fac']
# popups_list1 = list(zip(name1, source_fac1))
# popups1 = ["Name:{}<br>Facility:{}".format(Name, source_fac) for (Name, source_fac) in popups_list1]

# name2 = hospital1['Name']
# source_fac2 = hospital1['source_fac']
# popups_list2 = list(zip(name2, source_fac2))
# popups2 = ["Name:{}<br>Facility:{}".format(Name, source_fac) for (Name, source_fac) in popups_list2]

# name3 = residential1['Name']
# source_fac3 = residential1['source_fac']
# popups_list3 = list(zip(name3, source_fac3))
# popups3 = ["Name:{}<br>Facility:{}".format(Name, source_fac) for (Name, source_fac) in popups_list3]

# name4 = others1['Name']
# source_fac4 = others1['source_fac']
# popups_list4 = list(zip(name4, source_fac4))
# popups4 = ["Name:{}<br>Facility:{}".format(Name, source_fac) for (Name, source_fac) in popups_list4]

map1[map1['odhf_facil'] == 'Hospitals'].explore(m = base, #pass the base map object
             marker_kwds=dict(radius=5, fill=True), 
             legend = True, show = False, #disable the showing of the marker when map first loads
             color='#4654A3', popup = 'Facility Information',
             tooltip="Name", name = 'Hospital Locations')


map1[map1['odhf_facil'] == 'Nursing and residential care facilities'].explore(m = base,
             marker_kwds=dict(radius= 5, fill=True), 
             show = False, #disable the showing of the marker when map first loads
             color='#3DD8F9', popup = 'Facility Information',
             tooltip="Name", name = 'Nursing and residential care facility Locations')


map1[map1['odhf_facil'] == 'Ambulatory health care services'].explore(m = base,
             marker_kwds=dict(radius=5, fill=True),
             color='#FEB7C7', popup = 'Facility Information',
             show = False, #disable the showing of the marker when map first loads
             tooltip="Name", name = 'Ambulatory health care service locations')

map1[map1['odhf_facil'] == 'Others'].explore(m = base,
             marker_kwds=dict(radius=5, fill=True),
             color='red', popup = 'Facility Information',
             show = False, #disable the showing of the marker when map first loads
             tooltip="Name", name = 'Other facility location')


folium.TileLayer('openstreetmap', control=True).add_to(base)  # use folium to add alternative tiles
 
folium.LayerControl(collapsed = False).add_to(base)  # use folium to add layer control

## Saving the map as html

outfp = "MentalHealthcareAccessMap.html"
base.save(outfp)
